In [1]:

from dotenv import dotenv_values
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains import LLMChain
from langchain.schema import HumanMessage, SystemMessage, AIMessage

# Load API key from .env
env_values = dotenv_values("./app.env")
openai_api_key = env_values.get("OPENROUTER_API_KEY")



# Initialize the model , openai/gpt-4o-mini
llm = ChatOpenAI(
    model_name="openai/gpt-oss-20b:free",
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=openai_api_key,
)


Example of format_instructions

In [2]:
format_instructions = "Replay with a datetime format DAY/MONTH/YEAR like `23/05/1988`."

prompt = PromptTemplate(
    template="when was {person} born?\n{format_instructions}",
    input_variables=['person'],
    partial_variables={"format_instructions": format_instructions}
)

message = prompt.format(person='thomas edison')

print(llm([HumanMessage(content=message)]))


content='11/02/1847' additional_kwargs={} example=False


PydanticOutputParser

In [ ]:
from pydantic import BaseModel, Field
from typing import List
from langchain.output_parsers import PydanticOutputParser

class Plate(BaseModel):
    plate_name: str = Field(description="name of the plat")
    ingredients: List[str] = Field(description="list of names of the ingredients")

parser = PydanticOutputParser(pydantic_object=Plate)

format_instructions = parser.get_format_instructions()

prompt = PromptTemplate(
    template="Answer the query.\n{plate_name}\n{format_instructions}",
    input_variables=['plate_name'],
    partial_variables={"format_instructions": format_instructions}
)

plate_name = 'Shawerma'

message = prompt.format(plate_name=plate_name)

response = llm([HumanMessage(content=message)])

print(response)


content='```json\n{\n  "plate_name": "Shawerma",\n  "ingredients": [\n    "sliced beef",\n    "onions",\n    "garlic",\n    "tahini sauce",\n    "pickles",\n    "lettuce",\n    "tomatoes",\n    "pita bread",\n    "olive oil",\n    "spices"\n  ]\n}\n```\n' additional_kwargs={} example=False


In [15]:
print(message)

Answer the query.
Shawerma
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"plate_name": {"title": "Plate Name", "description": "name of the plat", "type": "string"}, "ingredients": {"title": "Ingredients", "description": "list of names of the ingredients", "type": "array", "items": {"type": "string"}}}, "required": ["plate_name", "ingredients"]}
```


In [10]:
print(response.content)  # Access the content attribute

```json
{
  "plate_name": "Shawerma",
  "ingredients": [
    "sliced beef",
    "onions",
    "garlic",
    "tahini sauce",
    "pickles",
    "lettuce",
    "tomatoes",
    "pita bread",
    "olive oil",
    "spices"
  ]
}
```



In [14]:
output = parser.parse(response.content)  # Parse the content, not the full response
print(type(output.ingredients))

<class 'list'>


In [12]:
print(f"Plate name: {output.plate_name}")
print(f"Ingredients: {output.ingredients}")

Plate name: Shawerma
Ingredients: ['sliced beef', 'onions', 'garlic', 'tahini sauce', 'pickles', 'lettuce', 'tomatoes', 'pita bread', 'olive oil', 'spices']
